<a href="https://colab.research.google.com/github/ovokojo/fintequity/blob/main/How%20to%20Analyze%20Stocks%20with%20AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README
This notebook aims to explore possibilities of AI in investment analysis. It uses various APIs to pull data and perform analyses. To ensure a smooth experience, please follow the setup instructions below.

**Pre-requisites**

Before running this notebook, you will need access to the following APIs:


*   **SEC-API** for accessing financial statements and other disclosures from the U.S. Securities and Exchange Commission. Get a free API key at: https://sec-api.io/

*   **OpenAI API** for utilizing advanced AI models for data analysis, summaries, and more. Get an API key at: https://platform.openai.com/



**Getting Started**

Configure API Keys: Store your SEC-API and OpenAI API keys in a secure location. This notebook uses Colab Secrets.

Run the notebook: Execute the cells in order, making sure to input your API keys or modify the code as neccessary to access them.



In [23]:
!pip install -q sec-api
!pip install openai

from google.colab import userdata
import openai
from openai import OpenAI
import requests

client = OpenAI(
    api_key=userdata.get("OAI_KEY"),
)
SEC_KEY = userdata.get("SEC_KEY")

In [39]:
from enum import Enum

# Supported 10-K Section Items
class SectionFrom10K(Enum):
    BUSINESS = "1"
    RISK_FACTORS = "1A"
    UNRESOLVED_STAFF_COMMENTS = "1B"
    PROPERTIES = "2"
    LEGAL_PROCEEDINGS = "3"
    MINE_SAFETY_DISCLOSURES = "4"
    MARKET_FOR_REGISTRANT_COMMON_EQUITY = "5"
    SELECTED_FINANCIAL_DATA = "6"
    MANAGEMENT_DISCUSSION = "7"
    QUANTITATIVE_AND_QUALITATIVE_DISCLOSURES = "7A"
    FINANCIAL_STATEMENTS = "8"
    CHANGES_IN_DISAGREEMENTS = "9"
    CONTROLS_AND_PROCEDURES = "9A"
    OTHER_INFORMATION = "9B"
    DIRECTORS_EXECUTIVE_OFFICERS = "10"
    EXECUTIVE_COMPENSATION = "11"
    SECURITY_OWNERSHIP = "12"
    CERTAIN_RELATIONSHIPS = "13"
    PRINCIPAL_ACCOUNTANT_FEES = "14"
    EXHIBITS_AND_FINANCIAL_STATEMENT_SCHEDULES = "15"

    def __str__(self):
        return f"{self.value}"

In [25]:
# Query API - Search and filter all 18+ million filings and exhibits published on SEC EDGAR since 1993
def getLatest10K(ticker: str):
    """
    Retrieves the most recent Form 10-K filing for a given stock ticker symbol.

    Args:
    - ticker (str): The stock ticker symbol of the company for which the Form 10-K is requested.

    Returns:
    - dict: A dictionary containing the JSON response with the latest Form 10-K filing
            information if the request is successful (HTTP status code 200).
    - int: The HTTP status code of the response if the request is unsuccessful.
    """

    endpoint = f"https://api.sec-api.io?token={SEC_KEY}"
    params = {
        "query": {
          "query_string": {
            "query":  f"ticker:{ticker} AND formType:\"10-K\""
          }
        },
        "from": "0",
        "size": "1",
        "sort": [{ "filedAt": { "order": "desc" }}]
    }
    response = requests.post(endpoint, json=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)
        return response.status_code

# Example
# latest10K = getLatest10K('TSLA')

In [26]:
# Extractor API - Extract any text section from 10-Q, 10-K, and 8-K SEC filings.
def getLatest10kSection(ticker: str, section: str):
    """
    Retrieves a specified section from the latest 10-K filing of any stock using the SEC Query API & the extractor API

    Args:
        ticker (str): The stock ticker symbol of the company for which the Form 10-K is requested.
        section (str): Specific section of the filing to retrieve (e.g., "1A" for the Risk Factors section).
        10-K supported section codes: 1, 1A, 1B, 2, 3, 4, 5, 6, 7, 7A, 8, 9, 9A, 9B, 10, 11, 12, 13, 14, 15

    Returns:
        text or int: The extracted content in JSON format if request is successful; otherwise, the HTTP status code.
    """

    latest10K = getLatest10K(ticker)
    url_10k = latest10K['filings'][0]['linkToFilingDetails']

    endpoint = "https://api.sec-api.io/extractor"

    params = {
        "token": SEC_KEY,
        "url": url_10k,
        "item": section,
        "type": "text"
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        return response.text
    else:
        print(f"Request failed with code: {response.status_code}")
        return response.status_code

# Example
# getLatest10kSection("TSLA", "1")

In [47]:
# Next steps: analyze & summarize section with AI
def askAssistant(ticker: str, section: SectionFrom10K, prompt: str):
  data = getLatest10kSection(ticker, section)
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": f"{prompt}. Use {section.value} of {ticker} 10-K here: {data}",
          }
      ],
      model="gpt-3.5-turbo",
  )
  response = chat_completion.choices[0].message.content
  print(response)

# Example
askAssistant("COCO", SectionFrom10K.BUSINESS, "Give me a summary of the business")

The Vita Coco Company is a leading platform for brands in the functional beverage category. They offer a range of products including packaged coconut water, coconut oil, juice, milk, and protein-infused fitness drinks. The company operates globally, with primary markets in North America, the United Kingdom, and China. They also supply private label products to retailers and generate revenue from bulk sales to beverage and food companies. Vita Coco is committed to delivering natural, nutritious products that are better for consumers and the world, with a focus on sustainability and ethical values. They operate as a public benefit corporation, balancing profit and purpose to meet high standards of social and environmental performance. The company also supports coconut farming communities through charitable initiatives and partnerships. With a diverse workforce and a focus on diversity, equity, and inclusion, The Vita Coco Company places a strong emphasis on their employees and their comm

In [32]:
# Example use case: get executive summaries of multiple companies.
tickers = ["ABNB","EXPE","BKNG"]

summaries = []

for ticker in tickers:
  askAssistant(ticker, "1", "Give me a brief summary of the business model")
  print("\n")

Airbnb is a community-based platform focused on connection and belonging. The business model revolves around connecting hosts who offer unique stays and experiences with guests looking for more authentic and immersive travel experiences. The platform serves five stakeholders: employees, shareholders, hosts, guests, and the communities in which they live. The company's long-term growth strategy includes making hosting mainstream, growing and engaging the guest community, perfecting the core service, and expanding the global network. The platform is supported by technology that enables seamless onboarding for hosts, engaging booking experiences for guests, and a robust system of trust and safety features. Airbnb also focuses on human capital management, sustainability efforts, and compliance with regulations and laws in various jurisdictions. The company faces competition from other online travel agencies, listing and meta search websites, hotel chains, property management companies, and